# Simple Boundary Operators Fermion Parity Check

Created 07/03/2025

Objectives:
* Pick a simple SPT case, investigate numerically found boundary operators, and compute fermion parity. Compare with exact results if needed.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [89]:
from functools import reduce
from operator import mul

from collections import defaultdict

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment,
    get_right_identity_environment_from_tp_tensor,
    multiply_transfer_matrices
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

from SPTOptimization.gradients import (
    expectation_gradient_from_environments_and_b_tensor
)

In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/simple_fermionic_cluster_200_site_dmrg/"

In [8]:
def parse_file_name(file_name):
    non_trivial_proj_rep = bool(int(file_name[3]))

    n1_01 = int(file_name[8])
    n1_10 = int(file_name[9])

    return (non_trivial_proj_rep, (n1_01, n1_10))

In [9]:
loaded_data = dict()

for local_file_name in os.listdir(DATA_DIR):
    f_name = r"{}/{}".format(DATA_DIR, local_file_name, ignore_unknown=False)
    with h5py.File(f_name, 'r') as f:
        data = hdf5_io.load_from_hdf5(f)

        data_info = parse_file_name(local_file_name)
        loaded_data[data_info]=data['wavefunction']

In [10]:
loaded_data

{(False, (1, 1)): <tenpy.networks.mps.MPS at 0x137ae6b90>,
 (True, (1, 1)): <tenpy.networks.mps.MPS at 0x17c092d10>,
 (True, (0, 1)): <tenpy.networks.mps.MPS at 0x17c0f3710>,
 (False, (0, 1)): <tenpy.networks.mps.MPS at 0x17d37d110>,
 (False, (1, 0)): <tenpy.networks.mps.MPS at 0x17c0666d0>,
 (True, (1, 0)): <tenpy.networks.mps.MPS at 0x17d3a6f50>,
 (True, (0, 0)): <tenpy.networks.mps.MPS at 0x17aed8d90>,
 (False, (0, 0)): <tenpy.networks.mps.MPS at 0x17d62d590>}

# Definitons

In [11]:
MAX_VIRTUAL_BOND_DIM = 8
MAX_INTERMEDIATE_VIRTUAL_BOND_DIM = 2*MAX_VIRTUAL_BOND_DIM
# MPO bond dim?
MAX_MPO_BOND_DIM = 50

SVD_CUTOFF = 1e-3

Define bosonic symmetries. Label by the group element added.

In [12]:
np_00 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

np_01 = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

np_10 = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

np_11 = np.array([
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [1, 0, 0, 0]
])

In [13]:
bosonic_np_symmetries = [
    np_00,
    np_01,
    np_10,
    np_11
]

In [14]:
npc_00 = to_npc_array(np_00)
npc_01 = to_npc_array(np_01)
npc_10 = to_npc_array(np_10)
npc_11 = to_npc_array(np_11)

Define "fermionic symmetries". Just identity and JW string.

In [15]:
np_I = np.array([
    [1, 0],
    [0, 1]
])

np_JW = np.array([
    [1, 0],
    [0, -1]
])

In [16]:
fermionic_np_symmetries = [np_I, np_JW]

In [17]:
fermionic_npc_symmetries = [
    to_npc_array(X) for X in fermionic_np_symmetries
]

In [18]:
symmetry_actions = [
    [[b, f] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [19]:
shifted_symmetry_actions = [
    [[f, b] for b in bosonic_np_symmetries]
    for f in fermionic_np_symmetries
]

In [20]:
cases = dict()

for k, psi in loaded_data.items():

    for i, l in enumerate(symmetry_actions):

        for j, s in enumerate(l):
            case = SymmetryActionWithBoundaryUnitaries(
                psi,
                [s[1],] + s*49,
                left_symmetry_index=51,
                left_boundary_unitaries=[np_00, np_I, np_00],
                right_boundary_unitaries=[np_00, np_I, np_00]
            )

            cases[(*k, i, j)] = case

In [21]:
for c in cases.values():
    c.compute_svd_approximate_expectation()

In [22]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [23]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Functions below

In [24]:
P_LEG_LABEL_REGEX_STRING = r"^p\d*$"
p_leg_pattern = re.compile(P_LEG_LABEL_REGEX_STRING)

def is_single_physical_leg_label(label):
    return bool(p_leg_pattern.match(label))

In [25]:
def is_grouped_physical_leg_label(leg_label):
    head, *body, tail = leg_label
    if (head != '(') or (tail != ')'):
        return False

    body = ''.join(body)
    legs = body.split('.')

    return all(is_single_physical_leg_label(l) for l in legs)

In [26]:
def is_physical_leg_label(label):
    out = (
        is_single_physical_leg_label(label)
        or is_grouped_physical_leg_label(label)
    )

    return out

In [27]:
def extract_single_physical_leg_label_from_tensor(b):
    out = next(
        l for l in b.get_leg_labels()
        if is_single_physical_leg_label(l)
    )

    return out

In [28]:
def conjugate_single_physical_leg_label(label):
    return label + '*'

In [29]:
def conjugate_grouped_physical_leg_label(label):
    _, *body, _ = label
    single_labels = ''.join(body).split('.')

    conjugate_labels = [
        conjugate_single_physical_leg_label(l)
        for l in single_labels
    ]

    conjugate_body = '.'.join(conjugate_labels)
    out = '(' + conjugate_body + ')'
    
    return out

In [30]:
def conjugate_leg_label(label):
    if is_single_physical_leg_label(label):
        return conjugate_single_physical_leg_label(label)
    elif is_grouped_physical_leg_label(label):
        return conjugate_grouped_physical_leg_label(label)
    else:
        raise ValueError

In [31]:
def get_physical_leg_labels(t):
    out = [
        ll for ll in t.get_leg_labels()
        if is_physical_leg_label(ll)
    ]

    return out

### Other

In [32]:
def contract_virtual_legs(tl, tr):
    return npc.tensordot(tl, tr, ['vR', 'vL'])

In [33]:
def combine_tensors(tensors):
    out = reduce(contract_virtual_legs, tensors)

    leg_labels = [
        extract_single_physical_leg_label_from_tensor(t)
        for t in tensors
    ]

    out = out.combine_legs(leg_labels)

    return out

In [34]:
def combine_b_tensors(b_tensors):
    renamed_tensors = [
        b.replace_label('p', f'p{i}')
        for i, b in enumerate(b_tensors)
    ]

    return combine_tensors(renamed_tensors)

In [35]:
def get_identity_operator(mps_tensor):
    p_leg_label = get_physical_leg_labels(mps_tensor)[0]
    p_leg = mps_tensor.get_leg(p_leg_label)
    p_leg_label_conj = conjugate_leg_label(p_leg_label)

    out = npc.diag(
        1,
        leg=p_leg,
        dtype='complex',
        labels=[p_leg_label, p_leg_label_conj]
    )

    return out

In [36]:
def svd_reduce_split_tensor(t, max_inner_dim=MAX_VIRTUAL_BOND_DIM,
                           normalise=True, svd_cutoff=SVD_CUTOFF):
    U, S, VH = npc.svd(
        t,
        compute_uv=True,
        inner_labels=['vR', 'vL'],
        cutoff=svd_cutoff
    )

    # Truncate tensors:
    U = U[:, :max_inner_dim]
    S = S[:max_inner_dim]
    VH = VH[:max_inner_dim, :]

    if normalise:
        new_norm = np.sqrt(np.sum(S**2))
        S = S/new_norm

    """
    leg = VH.get_leg('vL')

    schmidt_values = npc.diag(S, leg, labels=['vL', 'vR'])
    """

    return U, S, VH

In [37]:
def split_combined_b(b, leftmost_schmidt_values,
                     max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = b.split_legs()

    num_sites = t.ndim - 2

    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    out_bs = list()
    out_schmidt_values = list()

    current_left_schmidt_values = leftmost_schmidt_values

    for i, ll in enumerate(p_leg_labels[:-1]):
        # In case the bond dimension has been truncated. May need to add in a
        # case if have less schmidt values than the bond dim...
        bond_dim = t.get_leg('vL').get_block_sizes()[0]
        t.iscale_axis(current_left_schmidt_values[:bond_dim], axis='vL')

        tail_legs = p_leg_labels[(i+1):]
        
        t = t.combine_legs([['vL', ll], ['vR', *tail_legs]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=True
        )

        bl = (
            U
            .split_legs()
            .replace_label(ll, 'p')
        )
        bl.iscale_axis(1/current_left_schmidt_values[:bond_dim], axis='vL')
        bl.iscale_axis(S, axis='vR')
        bl.itranspose(['vL', 'p', 'vR'])
        out_bs.append(bl)

        out_schmidt_values.append(S)
        current_left_schmidt_values=S

        t = VH.split_legs()

    bl = t.replace_label(p_leg_labels[-1], 'p')
    bl.itranspose(['vL', 'p', 'vR'])
    out_bs.append(bl)

    return out_bs, out_schmidt_values

In [38]:
def split_b(b, max_virtual_bond_dim=MAX_INTERMEDIATE_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    leg_label = get_physical_leg_labels(b)[0]

    if is_single_physical_leg_label(leg_label):
        return b
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_b(b, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [39]:
def split_combined_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
                     p_leg_labels=None):
    t = u.split_legs()
    num_sites = t.ndim //2
    
    if p_leg_labels is None:
        p_leg_labels = [f'p{i}' for i in range(num_sites)]

    p_leg_labels_hc = [l+'*' for l in p_leg_labels]
    out_ws = list()

    for i, (ll, ll_hc) in enumerate(zip(p_leg_labels[:-1], p_leg_labels_hc[:-1])):
        tail_legs = p_leg_labels[(i+1):]
        tail_legs_hc = p_leg_labels_hc[(i+1):]

        left_leg_labels = [ll, ll_hc]
        if 'vL' in t.get_leg_labels():
            left_leg_labels.append('vL')
    
        t = t.combine_legs([left_leg_labels, [*tail_legs, *tail_legs_hc]])

        U, S, VH = svd_reduce_split_tensor(
            t,
            max_inner_dim=max_virtual_bond_dim,
            normalise=False
        )

        U.iscale_axis(S, axis='vR')

        wl = U.split_legs().replace_labels([ll, ll_hc], ['p', 'p*'])
        #wl.itranspose(['p', 'p*'])
        out_ws.append(wl)

        t = VH.split_legs()

    wl = t.replace_labels(
        [p_leg_labels[-1], p_leg_labels_hc[-1]],
        ['p', 'p*']
    )
    #wl.itranspose(['p', 'p*'])
    out_ws.append(wl)

    out_ws[0] = (
        out_ws[0]
        .add_leg(left_trivial_leg_charge, 0, label='vL')
    )
    
    out_ws[-1] = (
        out_ws[-1]
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out_ws

In [40]:
def split_u(u, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM,
            p_leg_labels=None):
    leg_label = get_physical_leg_labels(u)[0]

    if is_single_physical_leg_label(leg_label):
        return [unitary_to_mpo_tensor(u),]
    elif is_grouped_physical_leg_label(leg_label):
        return split_combined_u(u, max_virtual_bond_dim, p_leg_labels)
    else:
        raise ValueError

In [41]:
def unitary_to_mpo_tensor(u):
    out = (
        u
        .add_leg(left_trivial_leg_charge, 0, label='vL')
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out

In [42]:
def multiply_individual_mpo_tensors(w1, w2):
    w1 = w1.replace_labels(['vR', 'vL'], ['vR1', 'vL1'])
    w2 = w2.replace_labels(['vR', 'vL'], ['vR2', 'vL2'])

    w = (
        npc.tensordot(w1, w2, [['p'], ['p*']])
        .combine_legs([['vR1', 'vR2'], ['vL1', 'vL2']])
        .replace_labels(['(vR1.vR2)', '(vL1.vL2)'], ['vR', 'vL'])
    )

    return w

In [43]:
def group_elements(l, group_size, offset=0):
    first, rest = l[:offset], l[offset:]

    num_rest_groups = ((len(rest)-1)//group_size) + 1

    groups = [first,] if first else list()

    for i in range(num_rest_groups):
        first_index = i*group_size
        last_index = (i+1)*group_size
        groups.append(rest[first_index:last_index])

    return groups

In [44]:
def combine_grouped_b_tensors(grouped_bs):
    out = list()

    for group in grouped_bs:
        if len(group) == 1:
            out.append(group[0])
        else:
            out.append(combine_b_tensors(group))

    return out

In [45]:
def reduce_virtual_mpo_dim(wl, wr, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    # wl = wl.replace_labels(['p', 'p*'], ['pl', 'pl*'])

    wl = wl.combine_legs(['p', 'vL', 'p*'])
    wr = wr.combine_legs(['p', 'vR', 'p*'])
    w = npc.tensordot(wl, wr, ['vR', 'vL'])

    wl, S, wr = svd_reduce_split_tensor(
        w,
        max_inner_dim=max_virtual_bond_dim
    )

    wl.iscale_axis(S, axis='vR')

    wl = wl.split_legs()
    wr = wr.split_legs()

    return wl, wr

In [46]:
def multiply_mpos(top_ws, bottom_ws, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    ws = list()
    
    for w1, w2 in zip(top_ws, bottom_ws):
        w = multiply_individual_mpo_tensors(w1, w2)
        ws.append(w)


    for i in range(len(ws)-1):
        ws[i], ws[i+1] = reduce_virtual_mpo_dim(
            ws[i],
            ws[i+1],
            max_virtual_bond_dim=max_virtual_bond_dim
        )

    return ws

### Layer function

In [47]:
def optimise_layer(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values

In [48]:
def inner_product_b_tensors(b_tensors, b_bra_tensors=None, left_environment=None,
                            right_environment=None):
    if b_bra_tensors is None:
        b_bra_tensors = b_tensors

    b = b_tensors[0]
    b_bra = b_bra_tensors[0]

    if left_environment is None:
        t = npc.tensordot(b, b_bra.conj(), [['vL',], ['vL*',]])
    else:
        t = npc.tensordot(left_environment, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    for b, b_bra in zip(b_tensors[1:], b_bra_tensors[1:]):
        t = npc.tensordot(t, b, [['vR',], ['vL',]])
        t = npc.tensordot(t, b_bra.conj(), [['vR*', 'p'], ['vL*', 'p*']])

    if right_environment is None:
        out = npc.trace(t)
    else:
        out = npc.tensordot(t, right_environment, [['vR', 'vR*'], ['vL', 'vL*']])

    return out

In [49]:
def get_left_side_right_symmetry_environment(
    right_top_b_tensors, right_bottom_b_tensors, symmetry_transfer_matrix
    ):

    if right_bottom_b_tensors is None:
        right_bottom_b_tensors = right_top_b_tensors

    t = get_right_identity_environment_from_tp_tensor(right_top_b_tensors[-1])

    for tb, bb in zip(right_top_b_tensors[::-1], right_bottom_b_tensors[::-1]):
        t = npc.tensordot(t, tb, [['vL',], ['vR']])
        t = npc.tensordot(t, bb.conj(), [['vL*', 'p'], ['vR*', 'p*']])

    t = npc.tensordot(
        t,
        symmetry_transfer_matrix,
        [['vL', 'vL*',], ['vR', 'vR*']]
    )

    return t

In [50]:
def swap_left_right_indices(npc_array):
    left_right_pairs = {
        'vL': 'vR',
        'vR': 'vL',
        'vL*': 'vR*',
        'vR*': 'vL*'
    }

    leg_labels = npc_array.get_leg_labels()

    old_labels = [l for l in leg_labels if l in left_right_pairs]
    new_labels = [left_right_pairs[l] for l in old_labels]

    out = npc_array.replace_labels(old_labels, new_labels)

    return out

In [51]:
def two_sided_optimise_layer(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_pair = optimise_layer(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_pair[0]
    new_right_side_left_schmidt_values = new_right_pair[1]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_pair = optimise_layer(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_pair[0]
    new_left_side_right_schmidt_values = new_left_pair[1]

    return (new_left_pair, new_right_pair)

In [52]:
def initialize_brick_optimisation(symmetry_case, num_sites):
    symmetry_transfer_matrix = symmetry_case.npc_symmetry_transfer_matrix

    right_site_indices = list(range(
        symmetry_case.right_symmetry_index + 1,
        symmetry_case.right_symmetry_index + 1 + num_sites
    ))

    right_mps_tensors = [
        symmetry_case.psi.get_B(i)
        for k, i in enumerate(right_site_indices)
    ]

    right_side_left_schmidt_values = [
        symmetry_case.psi.get_SL(i)
        for i in right_site_indices
    ]

    left_site_indices = list(range(
        symmetry_case.left_symmetry_index - 1,
        symmetry_case.left_symmetry_index - 1 - num_sites,
        -1
    ))

    left_mps_tensors = [
        symmetry_case.psi.get_B(i, form='A')
        for k, i in enumerate(left_site_indices)
    ]
    
    left_mps_tensors = [swap_left_right_indices(b) for b in left_mps_tensors]

    left_side_right_schmidt_values = [
        symmetry_case.psi.get_SR(i)
        for i in left_site_indices
    ]

    right_expectations=list()
    left_expectations=list()
    left_unitaries=list()
    right_unitaries=list()

    return (
        symmetry_transfer_matrix,
        left_mps_tensors,
        right_mps_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        right_expectations,
        left_expectations,
        left_unitaries,
        right_unitaries
    )

### Output gradient tensor "one site" functions

In [53]:
def optimise_layer_with_gradient(
        left_environment,
        top_b_tensors,
        left_schmidt_values,
        block_width,
        block_offset,
        expectations,
        unitaries=list(),
        bottom_b_tensors=None,
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_b_tensors is None:
        bottom_b_tensors = top_b_tensors

    group = lambda x: group_elements(x, block_width, block_offset)
    top_grouped_bs = group(top_b_tensors)
    bottom_grouped_bs = group(bottom_b_tensors)
    grouped_schmidt_values = group(left_schmidt_values)

    top_combined_bs = combine_grouped_b_tensors(top_grouped_bs)
    bottom_combined_bs = combine_grouped_b_tensors(bottom_grouped_bs)

    new_unitaries = [
        get_identity_operator(t) for t in top_combined_bs
    ]

    expectations.append(list())

    # Below code is a hack to add in the gradient, assuming one block.
    right_environment = get_right_identity_environment_from_tp_tensor(
        top_b_tensors[-1]
    )

    grad = expectation_gradient_from_environments_and_b_tensor(
        top_combined_bs[0],
        left_environment,
        right_environment,
        bottom_combined_bs[0]
    )
    # Hack ends here.

    for _ in range(num_iterations):
        exps, *_ = one_site_optimization_sweep_right(
            left_environment,
            top_combined_bs,
            new_unitaries,
            bottom_combined_bs
        )

        expectations[-1].append(exps)

    unitaries.append(new_unitaries)

    for i, u in enumerate(new_unitaries):
        b = top_combined_bs[i]
        ll = get_physical_leg_labels(b)[0]
        llh = conjugate_leg_label(ll)
    
        new_b = npc.tensordot(b, u, [[ll,], [llh,]])
    
        top_combined_bs[i] = new_b

    new_top_bs = list()
    #new_left_schmidt_values = left_schmidt_values.copy()
    new_left_schmidt_values = list()

    for b, s in zip (top_combined_bs, grouped_schmidt_values):
        """
        print(b)
        print(s)
        print('\n')
        """
        leg_label = get_physical_leg_labels(b)[0]
        if is_single_physical_leg_label(leg_label):
            new_top_bs.append(b)
            new_left_schmidt_values.extend(s)
        elif is_grouped_physical_leg_label(leg_label):
            bs, schmidt_vals = split_combined_b(
                b,
                s[0],
                max_virtual_bond_dim
            )
            new_top_bs.extend(bs)
            new_left_schmidt_values.extend(s)

    """
    for i, b in enumerate(new_top_bs):
        top_b_tensors[i] = b
    """

    return new_top_bs, new_left_schmidt_values, grad

In [54]:
def two_sided_optimise_layer_with_gradient(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_triple = optimise_layer_with_gradient(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_triple[0]
    new_right_side_left_schmidt_values = new_right_triple[1]
    new_right_grad = new_right_triple[2]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_triple = optimise_layer_with_gradient(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_triple[0]
    new_left_side_right_schmidt_values = new_left_triple[1]
    new_left_grad = new_left_triple[2]

    return (new_left_triple, new_right_triple)

### Output calculated symmetry environments

In [55]:
def two_sided_optimise_layer_with_environments(
        symmetry_transfer_matrix,
        top_left_b_tensors,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        left_expectations,
        bottom_left_b_tensors=None,
        bottom_right_b_tensors=None,
        left_unitaries=list(),
        right_unitaries=list(),
        num_iterations=1,
        max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM
    ):

    if bottom_left_b_tensors is None:
        bottom_left_b_tensors = top_left_b_tensors

    if bottom_right_b_tensors is None:
        bottom_right_b_tensors = top_right_b_tensors

    right_side_left_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            top_left_b_tensors,
            bottom_left_b_tensors,
            swap_left_right_indices(symmetry_transfer_matrix)
        )
    )

    right_side_left_symmetry_environment = swap_left_right_indices(
        right_side_left_symmetry_environment
    )
    
    new_right_triple = optimise_layer_with_gradient(
        right_side_left_symmetry_environment,
        top_right_b_tensors,
        right_side_left_schmidt_values,
        block_width,
        block_offset,
        right_expectations,
        right_unitaries,
        bottom_right_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_right_b_tensors = new_right_triple[0]
    new_right_side_left_schmidt_values = new_right_triple[1]
    new_right_grad = new_right_triple[2]

    left_side_right_symmetry_environment = (
        get_left_side_right_symmetry_environment(
            new_top_right_b_tensors,
            bottom_right_b_tensors,
            symmetry_transfer_matrix
        )
    )

    left_side_right_symmetry_environment = swap_left_right_indices(
        left_side_right_symmetry_environment
    )

    new_left_triple = optimise_layer_with_gradient(
        left_side_right_symmetry_environment,
        top_left_b_tensors,
        left_side_right_schmidt_values,
        block_width,
        block_offset,
        left_expectations,
        left_unitaries,
        bottom_left_b_tensors,
        num_iterations,
        max_virtual_bond_dim
    )

    new_top_left_b_tensors = new_left_triple[0]
    new_left_side_right_schmidt_values = new_left_triple[1]
    new_left_grad = new_left_triple[2]

    return (
        (*new_left_triple, left_side_right_symmetry_environment),
        (*new_right_triple, right_side_left_symmetry_environment)
    )

# Analysis

## Test case

In [60]:
sub_cases = dict()

for k, case in cases.items():
    pr, n1, *k1 = k
    if (pr, n1) == (False, (1, 0)):
        sub_cases[tuple(k1)] = case

In [61]:
sub_cases

{(0,
  0): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3942d0>,
 (0,
  1): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3943d0>,
 (0,
  2): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3944d0>,
 (0,
  3): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3945d0>,
 (1,
  0): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3946d0>,
 (1,
  1): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3947d0>,
 (1,
  2): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3948d0>,
 (1,
  3): <SPTOptimization.SymmetryActionWithBoundaryUnitaries.SymmetryActionWithBoundaryUnitaries at 0x17d3949d0>}

In [62]:
test_case_1 = sub_cases[(1, 1)]
test_case_2 = sub_cases[(1, 2)]

Compute boundary operators.

In [63]:
block_width = 3
num_one_sided_iterations = 2
num_two_sided_iterations = 2
num_layers = 1
num_sites = 3

In [68]:
c = test_case_1

(
    symmetry_transfer_matrix,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_right_side_left_schmidt_values,
    current_left_side_right_schmidt_values,
    current_right_expectations,
    current_left_expectations,
    current_left_unitaries,
    current_right_unitaries
) = initialize_brick_optimisation(c, num_sites)

top_right_mps_tensors = [current_right_mps_tensors,]
top_left_mps_tensors = [current_left_mps_tensors,]

right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

for i in range(num_layers):
    block_offset = (block_width//2)*i % block_width

    for _ in range(num_two_sided_iterations):
        out = two_sided_optimise_layer_with_gradient(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_one_sided_iterations
        )

        left_triple, right_triple = out
    
        top_right_mps_tensors.append(right_triple[0])
        right_side_left_schmidt_values_list.append(right_triple[1])
        
        top_left_mps_tensors.append(left_triple[0])
        left_side_right_schmidt_values_list.append(left_triple[1])

right_expectations_1 = current_right_expectations
left_expectations_1 = current_left_expectations
left_unitaries_1 = current_left_unitaries
right_unitaries_1 = current_right_unitaries

In [69]:
c = test_case_2

(
    symmetry_transfer_matrix,
    current_left_mps_tensors,
    current_right_mps_tensors,
    current_right_side_left_schmidt_values,
    current_left_side_right_schmidt_values,
    current_right_expectations,
    current_left_expectations,
    current_left_unitaries,
    current_right_unitaries
) = initialize_brick_optimisation(c, num_sites)

top_right_mps_tensors = [current_right_mps_tensors,]
top_left_mps_tensors = [current_left_mps_tensors,]

right_side_left_schmidt_values_list = [current_right_side_left_schmidt_values,]
left_side_right_schmidt_values_list = [current_left_side_right_schmidt_values,]

for i in range(num_layers):
    block_offset = (block_width//2)*i % block_width

    for _ in range(num_two_sided_iterations):
        out = two_sided_optimise_layer_with_gradient(
            symmetry_transfer_matrix,
            top_left_mps_tensors[-1],
            top_right_mps_tensors[-1],
            right_side_left_schmidt_values_list[-1],
            left_side_right_schmidt_values_list[-1],
            block_width,
            block_offset,
            current_right_expectations,
            current_left_expectations,
            current_left_mps_tensors,
            current_right_mps_tensors,
            current_left_unitaries,
            current_right_unitaries,
            num_one_sided_iterations
        )

        left_triple, right_triple = out
    
        top_right_mps_tensors.append(right_triple[0])
        right_side_left_schmidt_values_list.append(right_triple[1])
        
        top_left_mps_tensors.append(left_triple[0])
        left_side_right_schmidt_values_list.append(left_triple[1])

right_expectations_2 = current_right_expectations
left_expectations_2 = current_left_expectations
left_unitaries_2 = current_left_unitaries
right_unitaries_2 = current_right_unitaries

### Expectations

In [70]:
right_expectations_2

[[[1.414146333677409e-14], [1.414146333677409e-14]],
 [[1.0000000000000033], [1.0000000000000033]]]

In [71]:
right_expectations_1

[[[1.2996392673343004e-17], [1.2996392673343004e-17]],
 [[1.0000000000000022], [1.0000000000000022]]]

In [72]:
left_expectations_1

[[[1.0000000000000033], [1.0000000000000033]],
 [[1.000000000000003], [1.000000000000003]]]

In [73]:
left_expectations_2

[[[0.9999999678482164], [0.9999999678482164]],
 [[1.0000000000000036], [1.0000000000000036]]]

### Unitary check.

In [75]:
right_unitary_2 = right_unitaries_2[-1][-1]

In [85]:
np.round(right_unitary_2.to_ndarray(), 2)

array([[ 0.85+0.j, -0.08+0.j, -0.07+0.j, ..., -0.  +0.j,  0.15+0.j,
         0.  +0.j],
       [-0.  +0.j, -0.05+0.j, -0.11+0.j, ..., -0.  +0.j, -0.06+0.j,
        -0.  +0.j],
       [-0.3 +0.j,  0.25+0.j,  0.43+0.j, ..., -0.  +0.j,  0.23+0.j,
         0.  +0.j],
       ...,
       [-0.  +0.j, -0.  +0.j, -0.  +0.j, ...,  0.55+0.j, -0.  +0.j,
        -0.19+0.j],
       [-0.02+0.j,  0.06+0.j,  0.12+0.j, ..., -0.  +0.j, -0.26+0.j,
        -0.  +0.j],
       [ 0.  +0.j, -0.  +0.j,  0.  +0.j, ...,  0.22+0.j, -0.  +0.j,
        -0.06+0.j]])

Not as clean a form as expected...

In [80]:
leg_labels = ['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*']

In [81]:
split_right_unitary_2 = (
    right_unitary_2
    .split_legs()
    .transpose(leg_labels)
)

In [82]:
split_right_unitary_2

<npc.Array shape=(4, 4, 2, 2, 4, 4) labels=['p0', 'p0*', 'p1', 'p1*', 'p2', 'p2*']>

In [87]:
np_split_right_unitary_2 = np.round(
    split_right_unitary_2
    .to_ndarray()
    ,2
)

In [88]:
non_zero_indcies = np.nonzero(np_split_right_unitary_2)

In [91]:
indices_dict = defaultdict(list)

for gul, gdl, gum, gdm, gur, gdr in zip(*non_zero_indcies):
    indices_dict[(gul, gdl, gur, gdr)].append((gum, gdm))

In [93]:
np_split_right_unitary_2[0, 0, ..., 0, 0]

array([[ 0.85+0.j, -0.09+0.j],
       [ 0.  +0.j,  0.36+0.j]])

In [94]:
np_split_right_unitary_2[0, 0, ..., 0, 1]

array([[-0.08+0.j,  0.01+0.j],
       [-0.08+0.j,  0.15+0.j]])

In [95]:
np_split_right_unitary_2[0, 0, ..., 0, 2]

array([[-0.07+0.j,  0.  +0.j],
       [-0.08+0.j, -0.  +0.j]])

In [92]:
indices_dict

defaultdict(list,
            {(0, 0, 0, 0): [(0, 0), (0, 1), (1, 1)],
             (0, 0, 0, 1): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 0, 2): [(0, 0), (1, 0)],
             (0, 0, 0, 3): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 1, 1): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 1, 2): [(0, 0), (1, 0)],
             (0, 0, 1, 3): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 2, 0): [(0, 0), (0, 1)],
             (0, 0, 2, 1): [(0, 0), (0, 1)],
             (0, 0, 2, 2): [(0, 0), (1, 1)],
             (0, 0, 2, 3): [(0, 0), (0, 1)],
             (0, 0, 3, 0): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 3, 1): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 3, 2): [(0, 0), (1, 0)],
             (0, 0, 3, 3): [(0, 0), (0, 1), (1, 0), (1, 1)],
             (0, 0, 1, 0): [(0, 1), (1, 0), (1, 1)],
             (0, 1, 0, 1): [(0, 0), (1, 0)],
             (0, 1, 0, 3): [(0, 0), (1, 0)],
             (0, 1, 1, 1): [(0, 0), (1, 0)],

In [96]:
np.sum(np.abs(
    np_split_right_unitary_2[:, :, 1, 1, :, :]
    + np_split_right_unitary_2[:, :, 0, 0, :, :]
))

41.120000000000005

In [97]:
np.sum(np.abs(
    np_split_right_unitary_2[:, :, 0, 1, :, :]
    + np_split_right_unitary_2[:, :, 1, 0, :, :]
))

35.83

These are comparable, and they really shouldn't be...

In [66]:
X = np_00[(...,)  + (np.newaxis,)*2]*np_JW[(np.newaxis,)*2 + (...,)]
X = X[(...,)  + (np.newaxis,)*2]*np_00[(np.newaxis,)*4 + (...,)]

In [67]:
boundary_jw = npc.Array.from_ndarray(
    X,
    legs_charges[:-2],
    dtype=np.complex128,
    labels=leg_labels
)

right_boundary_jw_3 = right_boundary_jw_3.combine_legs(
    [['p0', 'p1', 'p2'], ['p0*', 'p1*', 'p2*']]
)

NameError: name 'legs_charges' is not defined

In [420]:
def fermion_parity_operator_check_3(operator):
    op = (
        operator
        .transpose(['(p0.p1.p2)', '(p0*.p1*.p2*)'])
        .to_ndarray()
    )
    fp = right_boundary_jw_3.to_ndarray()

    transformed_op = fp @ op @ fp

    x = np.trace(transformed_op @ (op.T.conj()))
    phase = x/np.abs(x)

    assert np.isreal(phase)
    phase = np.real(phase)

    error = np.linalg.norm(
        phase*transformed_op - op,
        ord='fro'
    )

    return (phase, error)

In [ ]:
# Only works for one layer, one block unitaries...
left_phase_error_pair_1 = fermion_parity_operator_check_3(
    current_left_unitaries[-1][-1],
)
right_phase_error_pair_1 = fermion_parity_operator_check_3(
    current_right_unitaries[-1][-1]
)